In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/client/session.py:1793: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
xlsx_path  = 'augl.xlsx'
df = pd.read_excel(xlsx_path )

In [ ]:
df['Healthy'] = df['Healthy'].astype(str)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
image_dir = '/content/drive/MyDrive/aug'

In [ ]:
train_df['image_path'] = train_df['imagePath'].apply(lambda x: os.path.join(image_dir, x))
test_df['image_path'] = test_df['imagePath'].apply(lambda x: os.path.join(image_dir, x))


In [ ]:
train_df.head

<bound method NDFrame.head of              imagePath Healthy                                  image_path
5192   image0204_0.jpg       1  /content/drive/MyDrive/aug/image0204_0.jpg
4982   image0055_0.jpg       0  /content/drive/MyDrive/aug/image0055_0.jpg
9257   image2457_0.jpg       0  /content/drive/MyDrive/aug/image2457_0.jpg
11527  image3413_2.jpg       0  /content/drive/MyDrive/aug/image3413_2.jpg
4972   image0051_2.jpg       0  /content/drive/MyDrive/aug/image0051_2.jpg
...                ...     ...                                         ...
11964  image3582_1.jpg       0  /content/drive/MyDrive/aug/image3582_1.jpg
5191   image0203_2.jpg       0  /content/drive/MyDrive/aug/image0203_2.jpg
5390   image0273_0.jpg       0  /content/drive/MyDrive/aug/image0273_0.jpg
860      image0413.jpg       0    /content/drive/MyDrive/aug/image0413.jpg
7270   image0951_2.jpg       0  /content/drive/MyDrive/aug/image0951_2.jpg

[10240 rows x 3 columns]>

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def preprocess_image(img_path, label):
    # Load image
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0  # Normalize to [0,1]

    return img, label

def preprocess_image(img_path, label):
    try:
        # Load image
        img = tf.io.read_file(img_path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, (224, 224))
        img = tf.cast(img, tf.float32) / 255.0  # Normalize to [0,1]
        return img, label
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        return None, None

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='Healthy',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary'
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='Healthy',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary'
)


Found 7427 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 2813 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


Found 1814 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 746 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
# Print the first element of the train generator
print(next(train_generator))

# Print the first element of the test generator
print(next(test_generator))

(array([[[[110.       , 109.       , 107.       ],
         [110.       , 109.       , 107.       ],
         [110.       , 109.       , 107.       ],
         ...,
         [159.61644  , 158.61644  , 154.61644  ],
         [143.       , 142.       , 138.       ],
         [143.       , 142.       , 138.       ]],

        [[110.       , 109.       , 107.       ],
         [110.       , 109.       , 107.       ],
         [110.       , 109.       , 107.       ],
         ...,
         [160.76593  , 159.76593  , 155.76593  ],
         [144.92307  , 143.92307  , 139.92307  ],
         [143.       , 142.       , 138.       ]],

        [[110.       , 109.       , 107.       ],
         [110.       , 109.       , 107.       ],
         [110.       , 109.       , 107.       ],
         ...,
         [158.52827  , 157.52827  , 153.52827  ],
         [150.23752  , 149.23752  , 145.23752  ],
         [143.       , 142.       , 138.       ]],

        ...,

        [[157.       , 156.       , 1

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)


In [ ]:
model.fit(train_generator, epochs=50, validation_data=test_generator, callbacks=[early_stopping])


Epoch 1/50
186/465 [===========>..................] - ETA: 16:55 - loss: 0.8505 - accuracy: 0.5853

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7bee2c2eb0b0>
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "/

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f'Accuracy: {accuracy * 100:.2f}%')